In [6]:
!pip install ultralytics opencv-python -q
from ultralytics import YOLO
import pandas as pd
import os

In [7]:
# === CONFIGURATION ===
model_path = "/kaggle/input/yolov12trainingpayload/runs/detect/train/weights/best.pt"
image_dir = "/kaggle/input/vip-cup-test-payload-chunk-1/chunk_0001/RGB/images"

# === LOAD MODEL ===
model = YOLO(model_path)
rows = []

In [8]:
def run_inference_on_directory(model, image_dir):
    print(f"🚀 Running inference on: {image_dir}")
    
    try:
        results = model.predict(source=image_dir, imgsz=640, batch=16, device=0, stream=True)
    except Exception as e:
        print(f"❌ Error during inference: {e}")
        return []

    row_list = []
    for r in results:
        image_name = os.path.basename(r.path)

        for box in r.boxes:
            cls = int(box.cls)
            conf = float(box.conf)
            prob_harmful = conf if cls == 0 else 0.0
            payload_label = "Harmful" if cls == 0 else "Normal"

            xc, yc, w, h = box.xywhn[0]

            x_min = xc - w / 2
            y_min = yc - h / 2
            x_max = xc + w / 2
            y_max = yc + h / 2

            row_list.append({
                "Frame_name": image_name,
                "track_id": 0,
                "x_min_norm": f"{x_min:.4f}",
                "y_min_norm": f"{y_min:.4f}",
                "x_max_norm": f"{x_max:.4f}",
                "y_max_norm": f"{y_max:.4f}",
                "class_label": 0,
                "direction": 0,
                "confidence_detection": 0,
                "inference_time_detection (ms)": 0,
                "FPS (CPU)": 0,
                "FPS (GPU)": 0,
                "confidence_track": 0,
                "payload_label": payload_label,
                "prob_harmful": f"{prob_harmful:.4f}"
            })
    
    return row_list


In [9]:
# === RUN INFERENCE ===
rows = run_inference_on_directory(model, image_dir)

# === SAVE CSV ===
if rows:
    df = pd.DataFrame(rows)
    df.to_csv("submissionpayload.csv", index=False)
    print("✅ CSV saved as submissionpayload.csv")
else:
    print("⚠️ No detections found.")

🚀 Running inference on: /kaggle/input/vip-cup-test-payload-chunk-1/chunk_0001/RGB/images

image 1/2957 /kaggle/input/vip-cup-test-payload-chunk-1/chunk_0001/RGB/images/100_png.rf.30006a188acab12d1274058f7a0e5788v2.jpg: 640x640 1 Harmful, 6.6ms
image 2/2957 /kaggle/input/vip-cup-test-payload-chunk-1/chunk_0001/RGB/images/100_png.rf.30006a188acab12d1274058f7a0e5788v2_d1.jpg: 640x640 (no detections), 6.6ms
image 3/2957 /kaggle/input/vip-cup-test-payload-chunk-1/chunk_0001/RGB/images/100_png.rf.30006a188acab12d1274058f7a0e5788v2_d2.jpg: 640x640 1 Harmful, 6.6ms
image 4/2957 /kaggle/input/vip-cup-test-payload-chunk-1/chunk_0001/RGB/images/100_png.rf.30006a188acab12d1274058f7a0e5788v2_d3.jpg: 640x640 1 Harmful, 6.6ms
image 5/2957 /kaggle/input/vip-cup-test-payload-chunk-1/chunk_0001/RGB/images/100_png.rf.30006a188acab12d1274058f7a0e5788v2_v2.jpg: 640x640 1 Harmful, 6.6ms
image 6/2957 /kaggle/input/vip-cup-test-payload-chunk-1/chunk_0001/RGB/images/100_png.rf.30006a188acab12d1274058f7a0e5788v